In [ ]:
#!pip install openai

In [ ]:
from openai import OpenAI
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os

# OpenAI Settings
APIKey = ''

import openai
openai.api_key = APIKey


In [ ]:
#Knowledge base
knowledgebase = [
    # === Problemy techniczne (20) ===
    ("Problem z działaniem aplikacji mobilnej – aplikacja nie ładuje się lub nie wyświetla salda.", "Problemy techniczne"),
    ("Trudności z logowaniem do bankowości internetowej – brak dostępu do konta.", "Problemy techniczne"),
    ("Nie mogę autoryzować przelewu – aplikacja zawiesza się przy potwierdzaniu.", "Problemy techniczne"),
    ("Aplikacja bankowa przestała działać po aktualizacji.", "Problemy techniczne"),
    ("Nie działa funkcja skanowania kodów QR w aplikacji.", "Problemy techniczne"),
    ("Po włączeniu aplikacji pojawia się komunikat o błędzie serwera.", "Problemy techniczne"),
    ("Nie otrzymuję kodów SMS potrzebnych do logowania.", "Problemy techniczne"),
    ("Ekran logowania w aplikacji jest pusty, nic się nie wyświetla.", "Problemy techniczne"),
    ("Nie mogę dodać nowego urządzenia do autoryzacji.", "Problemy techniczne"),
    ("Aplikacja mobilna działa bardzo wolno po ostatniej aktualizacji.", "Problemy techniczne"),
    ("Nie działa funkcja przelewu natychmiastowego w aplikacji.", "Problemy techniczne"),
    ("Po zalogowaniu do aplikacji nie widać historii transakcji.", "Problemy techniczne"),
    ("Aplikacja zawiesza się podczas wykonywania przelewu.", "Problemy techniczne"),
    ("Nie mogę ustawić nowego PIN-u w aplikacji.", "Problemy techniczne"),
    ("Aplikacja nie rozpoznaje odcisku palca przy logowaniu.", "Problemy techniczne"),
    ("Po aktualizacji systemu Android aplikacja przestała działać.", "Problemy techniczne"),
    ("Nie mogę zaktualizować aplikacji bankowej w sklepie Google Play.", "Problemy techniczne"),
    ("Błąd połączenia z serwerem przy próbie wykonania przelewu.", "Problemy techniczne"),
    ("Zniknęły moje zapisane szablony przelewów w aplikacji.", "Problemy techniczne"),
    ("Aplikacja mobilna wylogowuje mnie automatycznie co kilka minut.", "Problemy techniczne"),

    # === Wnioski (20) ===
    ("Złożenie wniosku o kredyt gotówkowy – jakie dokumenty są potrzebne?", "Wnioski"),
    ("Prośba o zmianę oprocentowania lokaty terminowej.", "Wnioski"),
    ("Chciałbym otworzyć nowe konto oszczędnościowe – jaka jest procedura?", "Wnioski"),
    ("Jak złożyć wniosek o kartę kredytową online?", "Wnioski"),
    ("Czy mogę zwiększyć limit na karcie kredytowej?", "Wnioski"),
    ("Jak odnowić lokatę po zakończeniu okresu trwania?", "Wnioski"),
    ("Proszę o przesłanie formularza wniosku o kredyt hipoteczny.", "Wnioski"),
    ("Jak mogę przenieść rachunek oszczędnościowy do innego oddziału?", "Wnioski"),
    ("Wniosek o wydanie duplikatu karty płatniczej.", "Wnioski"),
    ("Jak złożyć wniosek o zmianę danych osobowych w umowie?", "Wnioski"),
    ("Czy mogę otworzyć konto dla dziecka przez Internet?", "Wnioski"),
    ("Jak odwołać złożony wcześniej wniosek o kartę kredytową?", "Wnioski"),
    ("Chcę zwiększyć limit kredytu odnawialnego na koncie osobistym.", "Wnioski"),
    ("Jak złożyć wniosek o kartę debetową do istniejącego konta?", "Wnioski"),
    ("Proszę o informację, jak złożyć wniosek o konsolidację kredytów.", "Wnioski"),
    ("Jak przedłużyć okres trwania lokaty terminowej?", "Wnioski"),
    ("Czy mogę zmienić typ konta z indywidualnego na wspólne?", "Wnioski"),
    ("Jak złożyć wniosek o wydanie zaświadczenia o posiadaniu rachunku?", "Wnioski"),
    ("Proszę o informację o wymaganiach do wniosku o leasing.", "Wnioski"),
    ("Jak złożyć wniosek o wakacje kredytowe?", "Wnioski"),

    # === Reklamacje (20) ===
    ("Niewłaściwie naliczone odsetki od kredytu – proszę o korektę.", "Reklamacje"),
    ("Brak realizacji przelewu – środki zostały pobrane, ale nie dotarły do odbiorcy.", "Reklamacje"),
    ("Bank pobrał podwójną opłatę za prowadzenie konta.", "Reklamacje"),
    ("Nieprawidłowo naliczona prowizja za wcześniejszą spłatę kredytu.", "Reklamacje"),
    ("Reklamacja błędnego salda – konto pokazuje zły stan środków.", "Reklamacje"),
    ("Pobrano prowizję za usługę, której nie zamawiałem.", "Reklamacje"),
    ("Reklamacja płatności kartą – transakcja została odrzucona, ale środki zniknęły.", "Reklamacje"),
    ("Naliczone odsetki nie zgadzają się z harmonogramem spłat.", "Reklamacje"),
    ("Nie otrzymałem zwrotu środków po anulowaniu transakcji.", "Reklamacje"),
    ("Podczas płatności kartą kwota została pobrana dwa razy.", "Reklamacje"),
    ("Bank odrzucił przelew mimo wystarczających środków na koncie.", "Reklamacje"),
    ("Nieprawidłowe naliczenie opłaty za obsługę karty kredytowej.", "Reklamacje"),
    ("Reklamacja dotycząca błędnego przeliczenia waluty przy płatności.", "Reklamacje"),
    ("Środki z konta zostały pobrane bez mojej zgody.", "Reklamacje"),
    ("Nie otrzymałem potwierdzenia złożonej reklamacji.", "Reklamacje"),
    ("Reklamacja błędnego kursu walutowego przy przewalutowaniu.", "Reklamacje"),
    ("Nieprawidłowa wysokość raty kredytu po zmianie oprocentowania.", "Reklamacje"),
    ("Brak informacji o odrzuconej reklamacji w aplikacji.", "Reklamacje"),
    ("Nie mogę sprawdzić statusu mojej reklamacji w systemie online.", "Reklamacje"),
    ("Bank pobrał opłatę za kartę, mimo że została zastrzeżona.", "Reklamacje"),

] 


In [ ]:
#declare connection with OpenAI
from openai import OpenAI
client = OpenAI(
    api_key=APIKey,
)

In [ ]:
# ==============================
# Generowanie embeddingów
# ==============================
def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response.data[0].embedding

In [ ]:
#Create a database 
import pandas as pd
embeddings = [get_embedding(text) for text in knowledgebase]

df["Embedding"] = df["Zgloszenie"].apply(get_embedding)

In [ ]:
df

## Data

In [ ]:
# ============================================
# Podział na zbiór treningowy i testowy
# ============================================

X = np.array(df["Embedding"].to_list())
y = df["Kategoria"]

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)


In [ ]:
# ============================================
# Trenowanie prostego modelu klasyfikacyjnego
# ============================================

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

In [ ]:
# ============================================
# Ewaluacja modelu
# ============================================

y_pred = clf.predict(X_test)

print("\n Raport klasyfikacji:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))

print("\n Macierz pomyłek:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# ============================================
# Test modelu na nowym zgłoszeniu
# ============================================

def predict_category(text):
    emb = get_embedding(text)
    pred = clf.predict([emb])[0]
    return encoder.inverse_transform([pred])[0]

In [ ]:
# ============================================
# Przykładowe zapytania testowe
# ============================================

test_texts = [
    "Nie mogę zalogować się do aplikacji na telefonie.",
    "Chciałbym złożyć wniosek o pożyczkę online.",
    "Przelew nie doszedł do adresata, mimo że środki zniknęły z konta.",
]

print("\n Przykładowe przewidywania:")
for t in test_texts:
    print(f" '{t}' → {predict_category(t)}")

In [ ]:
# ==============================
# Przykład użycia modelu
# ==============================
nowe_zgloszenie = "Nie mogę zalogować się do aplikacji mobilnej, pojawia się błąd."
nowe_emb = get_embedding(nowe_zgloszenie)
pred_kategoria = model.predict([nowe_emb])[0]

print("\nNowe zgłoszenie:", nowe_zgloszenie)
print("Przewidziana kategoria:", pred_kategoria)

In [ ]:
# ==============================
# (Opcjonalnie) Wizualizacja embeddingów w 2D
# ==============================
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
reduced = pca.fit_transform(X)
plt.figure(figsize=(8,6))
for cat in df["Zgloszenie"].unique():
    idx = df["Zgloszenie"] == cat
    plt.scatter(reduced[idx,0], reduced[idx,1], label=cat)
plt.legend()
plt.title("Wizualizacja embeddingów (PCA)")
plt.show()